In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
data_set = pd.read_csv('../../data/TrueFX/EUR-USD/datos_procesados',
                       index_col=0,infer_datetime_format=True, parse_dates=True)

In [3]:
data_set.head()

,open,high,low,bid,ask
2018-01-02 00:05:00,1.20094,1.20094,1.20011,1.20030,1.20115
2018-01-02 00:10:00,1.20030,1.20098,1.20011,1.20078,1.20123
2018-01-02 00:15:00,1.20073,1.20097,1.20032,1.20032,1.20094
2018-01-02 00:20:00,1.20030,1.20057,1.20025,1.20051,1.20086
2018-01-02 00:25:00,1.20052,1.20053,1.20048,1.20048,1.20083


In [4]:
%run ../../code/PNLEstimatorWrapper.py
%run ../../code/EUtilities.py
EU = EUtilities()

In [5]:
window = 12
X, y_reg, bt = EU.build_dataset(data_set, window=window, bid_col='bid', binary_target=True, PNL=True)

In [6]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,PNL_0,PNL_1
2018-01-02 01:00:00,1.20030,1.20078,1.20032,1.20051,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,0.00003,-0.00029
2018-01-02 01:05:00,1.20078,1.20032,1.20051,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,1.20099,-0.00017,-0.00011
2018-01-02 01:10:00,1.20032,1.20051,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,1.20099,1.20103,-0.00053,0.00030
2018-01-02 01:15:00,1.20051,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,1.20099,1.20103,1.20146,0.00004,-0.00019
2018-01-02 01:20:00,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,1.20099,1.20103,1.20146,1.20137,0.00012,-0.00023


In [7]:
X['open'] = data_set.filter(X.index,axis=0).open
X['high'] = data_set.filter(X.index,axis=0).high
X['low'] = data_set.filter(X.index,axis=0).low

In [8]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,PNL_0,PNL_1,open,high,low
2018-01-02 01:00:00,1.20030,1.20078,1.20032,1.20051,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,0.00003,-0.00029,1.20139,1.20155,1.20114
2018-01-02 01:05:00,1.20078,1.20032,1.20051,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,1.20099,-0.00017,-0.00011,1.20116,1.20125,1.20022
2018-01-02 01:10:00,1.20032,1.20051,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,1.20099,1.20103,-0.00053,0.00030,1.20099,1.20121,1.20072
2018-01-02 01:15:00,1.20051,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,1.20099,1.20103,1.20146,0.00004,-0.00019,1.20103,1.20159,1.20080
2018-01-02 01:20:00,1.20048,1.20061,1.20123,1.20134,1.20143,1.20133,1.20141,1.20117,1.20099,1.20103,1.20146,1.20137,0.00012,-0.00023,1.20146,1.20150,1.20135


In [9]:
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, Activation, LSTM, Embedding, Bidirectional, TimeDistributed

In [10]:
X.shape, bt.shape

((11893, 17), (11893,))

In [11]:
X1 = np.array(X[[col for col in X.columns if col!='PNL_0' and col!='PNL_1']])

In [12]:
scaler = MinMaxScaler(feature_range=(0, 1))
X1 = scaler.fit_transform(X1)

In [14]:
X1 = X1.reshape(X1.shape[0],1,X1.shape[1])
y = bt
X1.shape, y.shape

((11893, 1, 15), (11893,))

In [15]:
sum(y==3), sum(y==2),sum(y==1), sum(y==0)

(791, 754, 5175, 5173)

In [16]:
760/5170

0.1470019342359768

como es un modelo de multiclasificación toca utilizar categorical_crossentropy y para esta hay que pasar y en formato (n_filas,n_clases)

In [17]:
y1 = to_categorical(y,num_classes=4)

In [18]:
print(y1.shape)
y1

(11893, 4)


array([[ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.]])

In [19]:
X1.shape, y1.shape

((11893, 1, 15), (11893, 4))

In [37]:
np.random.seed(7)
model = Sequential()
model.add(LSTM(200,input_shape=(1,15)))
#model.add(Bidirectional(LSTM(32), input_shape=(1,X1.shape[1])))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### para cuadrar los pesos

In [38]:
sum(y==0)/sum(y==0), sum(y==0)/sum(y==1), sum(y==0)/sum(y==2), sum(y==0)/sum(y==3)

(1.0, 0.99961352657004832, 6.8607427055702921, 6.5398230088495577)

In [39]:
s_w = (y==0)*.06 + (y==1)*.06 + (y==2)*.44 + (y==3)*.44

In [40]:
#model.fit(X1, y1, epochs=10, batch_size=1000, class_weight={0:0.1,1:0.1,2:0.4,3:0.4},sample_weight=s_w)
model.fit(X1, y1, epochs=10, batch_size=100, class_weight={0:0.06, 1:0.06, 2:0.44, 3:0.44})

Epoch 1/10
11893/11893 [==============================] - 1s - loss: 0.1518 - acc: 0.1355     
Epoch 2/10
11893/11893 [==============================] - 1s - loss: 0.1517 - acc: 0.1359     
Epoch 3/10
11893/11893 [==============================] - 1s - loss: 0.1518 - acc: 0.1147     
Epoch 4/10
11893/11893 [==============================] - 1s - loss: 0.1517 - acc: 0.1113     
Epoch 5/10
11893/11893 [==============================] - 1s - loss: 0.1517 - acc: 0.1197     
Epoch 6/10
11893/11893 [==============================] - 1s - loss: 0.1516 - acc: 0.1100     
Epoch 7/10
11893/11893 [==============================] - 1s - loss: 0.1516 - acc: 0.1382     
Epoch 8/10
11893/11893 [==============================] - 1s - loss: 0.1516 - acc: 0.1184     
Epoch 9/10
11893/11893 [==============================] - 2s - loss: 0.1516 - acc: 0.1355     
Epoch 10/10
11893/11893 [==============================] - 2s - loss: 0.1516 - acc: 0.1208     


In [41]:
pre = model.predict_classes(X1)

11360/11893 [===========================>..] - ETA: 0s

In [42]:
print('target 0: ', sum(pre==0))
print('target 1: ', sum(pre==1))
print('target 2: ', sum(pre==2))
print('target 3: ', sum(pre==3))

target 0:  22
target 1:  4743
target 2:  0
target 3:  7128


In [43]:
pnl_0 = X.PNL_0
pnl_1 = X.PNL_1

In [44]:
r = sum((pre==1)*pnl_1 + (pre==0)*pnl_0)

In [45]:
r

-0.22922999999997806

In [176]:
model.save('2mejor_modelo.h5')

In [96]:
np.save(file='X_2018-1_2018-2',arr=X1)

In [169]:
# %load ../../code/step_validation.py
def step_validationRNN(model, X, y, n_epochs, n_batch_size, class_weight, cv):
    '''
    Recibe el estimador,X,y, y un generador cv con el cual hace la validación
    dependiendo que la configuración que este tenga
    '''
    import pandas as pd
    import numpy as np
    result = []
    for index in cv:
        pnl_1 = X.iloc[index[1]].PNL_1
        pnl_0 = X.iloc[index[1]].PNL_0
        X_pre = np.array(X.iloc[index[1]][[col for col in X.columns if col!='PNL_0' and col!='PNL_1']])
        X_pre = X_pre.reshape(X_pre.shape[0],X_pre.shape[1],1)
        
        X1 = np.array(X.iloc[index[0]][[col for col in X.columns if col!='PNL_0' and col!='PNL_1']])
        X1 = X1.reshape(X1.shape[0],X1.shape[1],1)
        y1 = to_categorical(np.array(y[index[0]]))
        
        model.fit(X1, y1, epochs=n_epochs, batch_size=n_batch_size, class_weight=class_weight, verbose=0)
        pre = model.predict_classes(X_pre)
        r = sum((pre==1)*pnl_1 + (pre==0)*pnl_0)
        
        result.append(r)
    return np.array(result)

In [170]:
# %load ../../code/v_split.py
# Se crea un generador "v_split" para utilizar como método de validación cruzada
def v_splitRNN(X, n_bdtrain, n_bdtest, mday):

    """"
    Hace un particionado del dataset, para tomar n_bdtrain días para entrenar
    y n_bdtest para probar, además, mday representa el paso de día a correr.
    X, dataframe, se necesita el indice de este para separar por días.
    n_bdtrain, número de bussines day utilizados para train.
    n_bdtest, número de bussines day utilizados para test.
    mday, días a correr para cada validación.
    
    """
    import pandas as pd
    import numpy as np
    from datetime import date
    start_day = 0
    
    #Divide el data set según días de train, test y cuanto se va moviendo
    bussines_day = []
    dates = pd.unique(X.index.date) #saco las fechas para luego tomar solo año-mes-día

    for i in dates: bussines_day.append(date.__format__(i,'%Y-%m-%d')) #lista de los bussines day
    
    intervals = []
    count = 0
    for i in bussines_day:
        f = len(X[i])-1 +count
        intervals.append([count,f])
        count = f+1
    
    for i in range(len(intervals)-n_bdtrain):
        yield(np.arange(intervals[start_day:start_day+n_bdtrain][0][0],
                        intervals[start_day:start_day+n_bdtrain][n_bdtrain-1][1]+1),
              np.arange(intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][0][0],
                        intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][n_bdtest-1][1]+1))
    
        start_day += mday
        if start_day+n_bdtest > len(intervals)-n_bdtrain:
            break

In [171]:
model = Sequential()

model.add(LSTM(64, input_shape=(X1.shape[1],1), return_sequences=True))
model.add(Dense(4,activation='softmax'))
model.add(LSTM(32))
#model.add(Dense(10, activation="relu", input_shape=(10,)))
model.add(Dense(4, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [172]:
step_validationRNN(model,X,y,n_epochs=200, n_batch_size=32,class_weight={0:0.06,1:0.06,2:0.44,3:0.44}
                   ,cv=v_splitRNN(X,4,1,1))

23/23 [==============================] - 0sETA: 0s


array([ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
        0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
       -0.0049 , -0.01014, -0.01373, -0.02131, -0.01349, -0.01058,
       -0.00172, -0.01477, -0.00813, -0.01124,  0.     ])

In [173]:
np.mean(np.array([ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
        0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
       -0.0049 , -0.01014, -0.01373, -0.02131, -0.01349, -0.01058,
       -0.00172, -0.01477, -0.00813, -0.01124,  0.     ]))

-0.00478304347826087

In [105]:
for i in v_splitRNN(X,4,1,1):
    print(X.iloc[i[0]].shape, X.iloc[i[1]].shape)

(1119, 15) (1, 15)
(845, 15) (279, 15)
(840, 15) (283, 15)
(845, 15) (281, 15)
(844, 15) (282, 15)
(1125, 15) (280, 15)
(1126, 15) (1, 15)
(844, 15) (280, 15)
(843, 15) (282, 15)
(843, 15) (288, 15)
(851, 15) (282, 15)
(1132, 15) (283, 15)
(1135, 15) (1, 15)
(854, 15) (282, 15)
(848, 15) (280, 15)
(846, 15) (282, 15)
(845, 15) (286, 15)
(1130, 15) (284, 15)
(1132, 15) (1, 15)
(853, 15) (281, 15)
(852, 15) (281, 15)
(847, 15) (287, 15)
(850, 15) (23, 15)


In [34]:
# %load ../../code/v_split.py
# Se crea un generador "v_split" para utilizar como método de validación cruzada
def v_split(X, n_bdtrain, n_bdtest, mday):

    """"
    Hace un particionado del dataset, para tomar n_bdtrain días para entrenar
    y n_bdtest para probar, además, mday representa el paso de día a correr.
    X, dataframe, se necesita el indice de este para separar por días.
    n_bdtrain, número de bussines day utilizados para train.
    n_bdtest, número de bussines day utilizados para test.
    mday, días a correr para cada validación.
    
    """
    import pandas as pd
    import numpy as np
    from datetime import date
    start_day = 0
    
    #Divide el data set según días de train, test y cuanto se va moviendo
    bussines_day = []
    dates = pd.unique(X.index.date) #saco las fechas para luego tomar solo año-mes-día

    for i in dates: bussines_day.append(date.__format__(i,'%Y-%m-%d')) #lista de los bussines day
    
    intervals = []
    count = 0
    for i in bussines_day:
        f = len(X[i])-1 +count
        intervals.append([count,f])
        count = f+1
    
    for i in range(len(intervals)-n_bdtrain):
        yield(np.arange(intervals[start_day:start_day+n_bdtrain][0][0],
                        intervals[start_day:start_day+n_bdtrain][n_bdtrain-1][1]+1),
              np.arange(intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][0][0],
                        intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][n_bdtest-1][1]+1))
    
        start_day += mday
        if start_day+n_bdtest > len(intervals)-n_bdtrain:
            break

In [47]:
# %load ../../code/step_validation.py
def step_validationRNN(estimator, X, y, cv, n_epochs, n_batch_size):
    '''
    Recibe el estimador,X,y, y un generador cv con el cual hace la validación
    dependiendo que la configuración que este tenga
    '''
    import numpy as np
    result = []
    for index in cv:
        estimator.fit(X.iloc[index[0]], y[index[0]], epochs=n_epochs, batch_size=n_batch_size)
        result.append(estimator.score(X.iloc[index[1]], y[index[1]]))
    return np.array(result)